# TFM

## Importación de librerías

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import toml
import random
import tensorflow as tf
from sklearn.metrics import (
    accuracy_score, 
    f1_score,
    recall_score,
    precision_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)

## Ejecución de código en GPU

In [39]:
import tensorflow as tf

# Comprobar la versión de TensorFlow
print("TensorFlow version:", tf.__version__)

# Listar todos los dispositivos físicos disponibles
physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    print("🟢 TensorFlow ha detectado la(s) siguiente(s) GPU(s):")
    for gpu in physical_devices:
        print(f'\t{gpu}')
else:
    print("🔴 No se ha detectado ninguna GPU. TensorFlow está utilizando la CPU.")

TensorFlow version: 2.10.0
🟢 TensorFlow ha detectado la(s) siguiente(s) GPU(s):
	PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Lectura de fichero

In [40]:
data_dict = {}
DATA_DIR = '../../data/'

for data_folder in os.listdir(DATA_DIR):
    *data_files, config_file = os.listdir(f'{DATA_DIR}/{data_folder}')

    leak_value = toml.load(f'{DATA_DIR}/{data_folder}/{config_file}')['tanks']['flow_value']

    train_dataframe, test_data = random.sample(data_files, 2)

    data_dict[data_folder] = {
        'leak_value': leak_value,
        'train_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{train_dataframe}').drop(columns=['Unnamed: 0']),
        'test_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{test_data}').drop(columns=['Unnamed: 0'])
    }

### Conjuntos de datos de entrenamiento

In [41]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"16932,24",-5795,"9003,66","20140,9","20147,8","6,9","194,79",0
1,2,"20147,8",-5795,"8487,37","22840,17","22824,12","-16,05","51,05",0
2,3,"22824,12",-6285,"4549,79","21088,91","21147,56","58,65","78,96",0
3,4,"21147,56",-6085,"6664,97","21727,53","21782,08","54,55","88,64",0
4,5,"21782,08",-5950,"3939,69","19771,77","19772,17","0,4","188,2",0
...,...,...,...,...,...,...,...,...,...
995,996,"17751,65",-5940,"3360,0","15171,65","15219,44","47,79","-101,62",1
996,997,"15219,44",-6110,"4412,01","13521,45","13522,52","1,07","-153,6",1
997,998,"13522,52",-6235,"7069,05","14356,57","14257,47","-99,1","-289,6",1
998,999,"14257,47",-6190,"6181,85","14249,32","14242,17","-7,15","-280,19",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22430,06","-6075,0","5976,32","22331,38","22376,0","44,62","-116,83",0
1,2,"22376,0","-6065,0","3965,07","20276,07","20290,81","14,74","-45,27",0
2,3,"20290,81","-5775,0","6982,55","21498,36","21513,87","15,51","-60,91",0
3,4,"21513,87","-6035,0","6956,14","22435,01","22426,74","-8,27","5,59",0
4,5,"22426,74","-6030,0","6377,94","22774,68","22817,15","42,47","36,97",0
...,...,...,...,...,...,...,...,...,...
995,996,"2107,2","-5828,98","6463,64","2741,86","2620,44","-121,42","-167,54",1
996,997,"2620,44","-5955,0","6142,6","2808,04","2771,02","-37,02","-143,07",1
997,998,"2771,02","-5896,0","3108,82","-16,16","25,62","41,78","-150,11",1
998,999,"25,62","-5592,26","6421,37","854,73","896,61","41,88","-83,97",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22143,22","-5810,0","7001,2","23334,42","23347,02","12,6","-90,27",0
1,2,"23347,02","-6240,0","7270,93","24377,95","24283,54","-94,41","-105,27",0
2,3,"24283,54","-6430,0","3650,03","21503,57","21510,13","6,56","-17,7",0
3,4,"21510,13","-5850,0","6940,97","22601,1","22644,04","42,94","27,91",0
4,5,"22644,04","-6025,0","5910,84","22529,88","22565,92","36,04","80,35",0
...,...,...,...,...,...,...,...,...,...
995,996,"20,99","-0,0","0,0","20,99","7,38","-13,61","5,7",1
996,997,"7,38","-0,0","0,0","7,38","2,99","-4,39","-5,43",1
997,998,"2,99","-0,0","0,0","2,99","14,37","11,38","9,91",1
998,999,"14,37","-0,0","0,0","14,37","23,46","9,09","10,89",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21399,05","-5790,0","5638,02","21247,07","21227,09","-19,98","63,76",0
1,2,"21227,09","-5895,0","5241,74","20573,83","20576,89","3,06","133,61",0
2,3,"20576,89","-6055,0","6999,43","21521,32","21504,09","-17,23","33,97",0
3,4,"21504,09","-5995,0","9004,73","24513,82","24667,5","153,68","232,55",0
4,5,"24667,5","-6005,0","5447,75","24110,25","24146,71","36,46","202,77",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,62","-0,0","0,0","2,62","19,24","16,62","5,27",1
996,997,"19,24","-0,0","0,0","19,24","14,88","-4,36","5,99",1
997,998,"14,88","-0,0","0,0","14,88","9,37","-5,51","-4,99",1
998,999,"9,37","-0,0","0,0","9,37","3,89","-5,48","-11,62",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"18449,36","-6030,0","6911,24","19330,6","19460,91","130,31","181,93",0
1,2,"19460,91","-6125,0","8022,82","21358,73","21278,01","-80,72","73,05",0
2,3,"21278,01","-6065,0","6947,29","22160,3","22222,18","61,88","147,32",0
3,4,"22222,18","-5980,0","7979,31","24221,49","24182,24","-39,25","-32,62",0
4,5,"24182,24","-6245,0","2862,55","20799,79","20848,11","48,32","85,87",0
...,...,...,...,...,...,...,...,...,...
995,996,"5,66","-0,0","0,0","5,66","7,3","1,64","-9,82",1
996,997,"7,3","-0,0","0,0","7,3","10,34","3,04","-0,58",1
997,998,"10,34","-0,0","0,0","10,34","12,28","1,94","-5,24",1
998,999,"12,28","-0,0","0,0","12,28","8,88","-3,4","2,08",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22143,65","-6055,0","6215,36","22304,01","22242,32","-61,69","34,11",0
1,2,"22242,32","-5940,0","6691,65","22993,97","22983,66","-10,31","42,41",0
2,3,"22983,66","-5885,0","6130,98","23229,64","23301,88","72,24","58,48",0
3,4,"23301,88","-6030,0","5035,77","22307,65","22270,88","-36,77","25,47",0
4,5,"22270,88","-6005,0","6518,15","22784,03","22794,42","10,39","13,96",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,74","-0,0","0,0","2,74","3,88","1,14","3,0",1
996,997,"3,88","-0,0","0,0","3,88","0,72","-3,16","-2,72",1
997,998,"0,72","-0,0","0,0","0,72","1,13","0,41","-11,02",1
998,999,"1,13","-0,0","0,0","1,13","2,14","1,01","-19,34",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"15886,17","-5900,0","5970,3","15956,47","15980,01","23,54","68,46",0
1,2,"15980,01","-5810,0","6050,74","16220,75","16175,65","-45,1","8,64",0
2,3,"16175,65","-5760,0","5917,51","16333,16","16413,96","80,8","17,3",0
3,4,"16413,96","-5970,0","6982,8","17426,76","17460,02","33,26","68,23",0
4,5,"17460,02","-6340,0","7929,64","19049,66","19111,41","61,75","52,84",0
...,...,...,...,...,...,...,...,...,...
995,996,"5,52","-0,0","0,0","5,52","0,47","-5,05","-4,84",1
996,997,"0,47","-0,0","0,0","0,47","5,79","5,32","0,68",1
997,998,"5,79","-0,0","0,0","5,79","14,14","8,35","-0,25",1
998,999,"14,14","-0,0","0,0","14,14","4,73","-9,41","2,66",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"19999,39","-5905,0","6970,79","21065,18","21102,9","37,72","109,13",0
1,2,"21102,9","-6155,0","6060,28","21008,18","20966,69","-41,49","59,33",0
2,3,"20966,69","-5975,0","2934,69","17926,38","17990,19","63,81","85,1",0
3,4,"17990,19","-6305,0","6950,94","18636,13","18679,74","43,61","127,56",0
4,5,"18679,74","-5940,0","7943,2","20682,94","20731,84","48,9","178,79",0
...,...,...,...,...,...,...,...,...,...
995,996,"5,73","-0,0","0,0","5,73","3,13","-2,6","-19,24",1
996,997,"3,13","-0,0","0,0","3,13","3,87","0,74","-3,1",1
997,998,"3,87","-0,0","0,0","3,87","5,88","2,01","5,53",1
998,999,"5,88","-0,0","0,0","5,88","2,22","-3,66","-1,13",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22682,46","-5995,0","5831,75","22519,21","22535,33","16,12","459,1",0
1,2,"22535,33","-5975,0","3507,95","20068,28","20135,89","67,61","514,24",0
2,3,"20135,89","-6250,0","8020,71","21906,6","21854,08","-52,52","421,71",0
3,4,"21854,08","-5985,0","8129,34","23998,42","23900,39","-98,03","282,93",0
4,5,"23900,39","-6030,0","5690,89","23561,28","23581,5","20,22","287,66",0
...,...,...,...,...,...,...,...,...,...
995,996,"6,96","-0,0","0,0","6,96","3,42","-3,54","-0,25",1
996,997,"3,42","-0,0","0,0","3,42","18,91","15,49","16,44",1
997,998,"18,91","-0,0","0,0","18,91","16,72","-2,19","9,09",1
998,999,"16,72","-0,0","0,0","16,72","1,44","-15,28","-0,25",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22379,58","-5785,0","5690,39","22284,97","22319,5","34,53","144,59",0
1,2,"22319,5","-5980,0","6499,32","22838,82","22799,39","-39,43","171,58",0
2,3,"22799,39","-6000,0","6305,86","23105,25","23145,46","40,21","213,48",0
3,4,"23145,46","-6095,0","5814,71","22865,17","22845,93","-19,24","195,21",0
4,5,"22845,93","-6145,0","6270,76","22971,69","22978,1","6,41","290,05",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,71","-0,0","0,0","2,71","2,61","-0,1","-0,48",1
996,997,"2,61","-0,0","0,0","2,61","5,93","3,32","1,85",1
997,998,"5,93","-0,0","0,0","5,93","13,8","7,87","5,27",1
998,999,"13,8","-0,0","0,0","13,8","6,1","-7,7","-0,73",1


### Conjunto de datos de prueba

In [42]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"14217,87",-6025,"8078,98","16271,85","16202,83","-69,02","29,76",0
1,2,"16202,83",-5915,"7993,5","18281,33","18287,13","5,8","103,63",0
2,3,"18287,13",-6005,"5997,67","18279,8","18294,11","14,31","27,16",0
3,4,"18294,11",-6085,"3013,11","15222,22","15202,29","-19,93","93,76",0
4,5,"15202,29",-5800,"7941,86","17344,15","17431,85","87,7","159,23",0
...,...,...,...,...,...,...,...,...,...
995,996,"15406,65",-6030,"4978,48","14355,13","14337,52","-17,61","61,26",1
996,997,"14337,52",-6000,"5721,51","14059,03","14009,35","-49,68","-44,29",1
997,998,"14009,35",-6275,"6313,19","14047,54","14181,99","134,45","161,32",1
998,999,"14181,99",-6150,"5914,01","13946,0","14007,4","61,4","181,88",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21149,68","-5960,0","5940,46","21130,14","21204,78","74,64","255,0",0
1,2,"21204,78","-6315,0","7021,73","21911,51","21862,93","-48,58","164,23",0
2,3,"21862,93","-5890,0","3218,08","19191,01","19194,42","3,41","149,26",0
3,4,"19194,42","-6160,0","6739,31","19773,73","19802,52","28,79","127,48",0
4,5,"19802,52","-5960,0","6979,79","20822,31","20853,99","31,68","148,24",0
...,...,...,...,...,...,...,...,...,...
995,996,"2228,36","-5465,58","4141,29","904,07","873,35","-30,72","-173,3",1
996,997,"873,35","-6090,0","6592,68","1376,03","1220,09","-155,94","-308,8",1
997,998,"1220,09","-5980,0","7181,83","2421,92","2463,95","42,03","-206,86",1
998,999,"2463,95","-5850,0","5758,71","2372,66","2400,72","28,06","-176,26",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"23654,87","-6005,0","4298,93","21948,8","21972,61","23,81","163,31",0
1,2,"21972,61","-6095,0","4622,96","20500,57","20617,61","117,04","322,76",0
2,3,"20617,61","-6175,0","9067,79","23510,4","23442,51","-67,89","221,84",0
3,4,"23442,51","-5990,0","3410,32","20862,83","20890,7","27,87","233,22",0
4,5,"20890,7","-5915,0","6810,38","21786,08","21803,84","17,76","215,53",0
...,...,...,...,...,...,...,...,...,...
995,996,"10,65","-0,0","0,0","10,65","30,77","20,12","26,54",1
996,997,"30,77","-0,0","0,0","30,77","0,47","-30,3","-3,4",1
997,998,"0,47","-0,0","0,0","0,47","4,08","3,61","0,87",1
998,999,"4,08","-0,0","0,0","4,08","1,82","-2,26","-3,71",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21287,51","-6070,0","7042,91","22260,42","22216,71","-43,71","188,2",0
1,2,"22216,71","-6065,0","4682,57","20834,28","20856,74","22,46","259,79",0
2,3,"20856,74","-6035,0","7234,09","22055,83","22134,3","78,47","272,27",0
3,4,"22134,3","-6120,0","5847,84","21862,14","22001,79","139,65","380,74",0
4,5,"22001,79","-5605,0","3757,23","20154,02","20186,77","32,75","279,92",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,31","-0,0","0,0","3,31","4,22","0,91","-5,75",1
996,997,"4,22","-0,0","0,0","4,22","14,3","10,08","7,99",1
997,998,"14,3","-0,0","0,0","14,3","12,66","-1,64","10,33",1
998,999,"12,66","-0,0","0,0","12,66","2,96","-9,7","-1,54",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17548,09","-6265,0","4958,31","16241,4","16245,87","4,47","160,94",0
1,2,"16245,87","-5725,0","6084,29","16605,16","16550,86","-54,3","43,48",0
2,3,"16550,86","-5880,0","6957,27","17628,13","17694,1","65,97","93,95",0
3,4,"17694,1","-6025,0","7912,37","19581,47","19651,53","70,06","229,9",0
4,5,"19651,53","-5880,0","5996,13","19767,66","19771,94","4,28","319,27",0
...,...,...,...,...,...,...,...,...,...
995,996,"9,49","-0,0","0,0","9,49","2,52","-6,97","-20,78",1
996,997,"2,52","-0,0","0,0","2,52","1,6","-0,92","-5,93",1
997,998,"1,6","-0,0","0,0","1,6","7,92","6,32","-9,84",1
998,999,"7,92","-0,0","0,0","7,92","1,37","-6,55","-12,18",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"16942,12","-5825,0","5491,42","16608,54","16673,87","65,33","187,29",0
1,2,"16673,87","-5825,0","6897,1","17745,97","17875,86","129,89","237,48",0
2,3,"17875,86","-5965,0","6950,25","18861,11","18894,66","33,55","253,19",0
3,4,"18894,66","-5970,0","7962,56","20887,22","20937,92","50,7","387,03",0
4,5,"20937,92","-6135,0","5924,27","20727,19","20818,82","91,63","502,42",0
...,...,...,...,...,...,...,...,...,...
995,996,"19,68","-0,0","0,0","19,68","8,25","-11,43","0,72",1
996,997,"8,25","-0,0","0,0","8,25","3,65","-4,6","-8,36",1
997,998,"3,65","-0,0","0,0","3,65","8,4","4,75","4,92",1
998,999,"8,4","-0,0","0,0","8,4","23,49","15,09","16,35",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"18109,75","-5940,0","6999,66","19169,41","19140,47","-28,94","290,5",0
1,2,"19140,47","-6320,0","7014,09","19834,56","19825,75","-8,81","346,46",0
2,3,"19825,75","-6145,0","7026,85","20707,6","20677,99","-29,61","326,59",0
3,4,"20677,99","-5805,0","4783,79","19656,78","19702,96","46,18","340,89",0
4,5,"19702,96","-6195,0","6592,28","20100,24","20021,69","-78,55","117,34",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,39","-0,0","0,0","3,39","8,94","5,55","-9,07",1
996,997,"8,94","-0,0","0,0","8,94","13,46","4,52","1,87",1
997,998,"13,46","-0,0","0,0","13,46","5,2","-8,26","4,92",1
998,999,"5,2","-0,0","0,0","5,2","13,18","7,98","-1,52",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"23056,29","-5975,0","3639,31","20720,6","20716,68","-3,92","334,01",0
1,2,"20716,68","-6150,0","7008,88","21575,56","21681,55","105,99","352,59",0
2,3,"21681,55","-6030,0","6463,74","22115,29","22138,84","23,55","354,54",0
3,4,"22138,84","-6330,0","4019,4","19828,24","19807,63","-20,61","339,9",0
4,5,"19807,63","-6315,0","10039,4","23532,03","23487,69","-44,34","121,91",0
...,...,...,...,...,...,...,...,...,...
995,996,"1,75","-0,0","0,0","1,75","13,26","11,51","10,31",1
996,997,"13,26","-0,0","0,0","13,26","4,6","-8,66","-5,13",1
997,998,"4,6","-0,0","0,0","4,6","9,65","5,05","6,75",1
998,999,"9,65","-0,0","0,0","9,65","6,1","-3,55","-3,36",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"23299,15","-5900,0","4046,57","21445,72","21419,26","-26,46","496,31",0
1,2,"21419,26","-6310,0","8439,67","23548,93","23527,4","-21,53","356,96",0
2,3,"23527,4","-6055,0","7093,15","24565,55","24485,13","-80,42","233,55",0
3,4,"24485,13","-5925,0","2956,05","21516,18","21525,33","9,15","216,75",0
4,5,"21525,33","-6095,0","6018,92","21449,25","21425,77","-23,48","121,03",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,16","-0,0","0,0","3,16","2,82","-0,34","-5,42",1
996,997,"2,82","-0,0","0,0","2,82","2,05","-0,77","1,14",1
997,998,"2,05","-0,0","0,0","2,05","1,39","-0,66","-3,8",1
998,999,"1,39","-0,0","0,0","1,39","10,38","8,99","-2,67",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"11544,4","-5565,0","7069,47","13048,87","12982,38","-66,49","118,8",0
1,2,"12982,38","-6155,0","6953,56","13780,94","13840,48","59,54","124,42",0
2,3,"13840,48","-6005,0","8211,32","16046,8","15937,63","-109,17","-18,99",0
3,4,"15937,63","-5915,0","8550,06","18572,69","18595,78","23,09","76,99",0
4,5,"18595,78","-5830,0","6389,87","19155,65","19072,47","-83,18","-24,06",0
...,...,...,...,...,...,...,...,...,...
995,996,"16,15","-0,0","0,0","16,15","6,29","-9,86","-11,87",1
996,997,"6,29","-0,0","0,0","6,29","14,36","8,07","4,38",1
997,998,"14,36","-0,0","0,0","14,36","2,38","-11,98","-20,25",1
998,999,"2,38","-0,0","0,0","2,38","4,75","2,37","-6,32",1


## Limpieza de datos

In [43]:
bad_formatted_fields = [
    'Volumen dep. almacenam. ini. (L)',
    'Llenado dep. almacenam. (L)',
    'Volumen dep. almacenam. fin. teor. (L)',
    'Volumen dep. almacenam. fin. (L)',
    'Variacion',
    'Variacion Acum.',
]

for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        for field in bad_formatted_fields:
            data_dict[case][dataset][field] = data_dict[case][dataset][field].str.replace(',', '.')
            data_dict[case][dataset][field] = pd.to_numeric(data_dict[case][dataset][field])

### Conjunto de datos de entrenamiento

In [44]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,16932.24,-5795,9003.66,20140.90,20147.80,6.90,194.79,0
1,2,20147.80,-5795,8487.37,22840.17,22824.12,-16.05,51.05,0
2,3,22824.12,-6285,4549.79,21088.91,21147.56,58.65,78.96,0
3,4,21147.56,-6085,6664.97,21727.53,21782.08,54.55,88.64,0
4,5,21782.08,-5950,3939.69,19771.77,19772.17,0.40,188.20,0
...,...,...,...,...,...,...,...,...,...
995,996,17751.65,-5940,3360.00,15171.65,15219.44,47.79,-101.62,1
996,997,15219.44,-6110,4412.01,13521.45,13522.52,1.07,-153.60,1
997,998,13522.52,-6235,7069.05,14356.57,14257.47,-99.10,-289.60,1
998,999,14257.47,-6190,6181.85,14249.32,14242.17,-7.15,-280.19,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22430.06,"-6075,0",5976.32,22331.38,22376.00,44.62,-116.83,0
1,2,22376.00,"-6065,0",3965.07,20276.07,20290.81,14.74,-45.27,0
2,3,20290.81,"-5775,0",6982.55,21498.36,21513.87,15.51,-60.91,0
3,4,21513.87,"-6035,0",6956.14,22435.01,22426.74,-8.27,5.59,0
4,5,22426.74,"-6030,0",6377.94,22774.68,22817.15,42.47,36.97,0
...,...,...,...,...,...,...,...,...,...
995,996,2107.20,"-5828,98",6463.64,2741.86,2620.44,-121.42,-167.54,1
996,997,2620.44,"-5955,0",6142.60,2808.04,2771.02,-37.02,-143.07,1
997,998,2771.02,"-5896,0",3108.82,-16.16,25.62,41.78,-150.11,1
998,999,25.62,"-5592,26",6421.37,854.73,896.61,41.88,-83.97,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22143.22,"-5810,0",7001.20,23334.42,23347.02,12.60,-90.27,0
1,2,23347.02,"-6240,0",7270.93,24377.95,24283.54,-94.41,-105.27,0
2,3,24283.54,"-6430,0",3650.03,21503.57,21510.13,6.56,-17.70,0
3,4,21510.13,"-5850,0",6940.97,22601.10,22644.04,42.94,27.91,0
4,5,22644.04,"-6025,0",5910.84,22529.88,22565.92,36.04,80.35,0
...,...,...,...,...,...,...,...,...,...
995,996,20.99,"-0,0",0.00,20.99,7.38,-13.61,5.70,1
996,997,7.38,"-0,0",0.00,7.38,2.99,-4.39,-5.43,1
997,998,2.99,"-0,0",0.00,2.99,14.37,11.38,9.91,1
998,999,14.37,"-0,0",0.00,14.37,23.46,9.09,10.89,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21399.05,"-5790,0",5638.02,21247.07,21227.09,-19.98,63.76,0
1,2,21227.09,"-5895,0",5241.74,20573.83,20576.89,3.06,133.61,0
2,3,20576.89,"-6055,0",6999.43,21521.32,21504.09,-17.23,33.97,0
3,4,21504.09,"-5995,0",9004.73,24513.82,24667.50,153.68,232.55,0
4,5,24667.50,"-6005,0",5447.75,24110.25,24146.71,36.46,202.77,0
...,...,...,...,...,...,...,...,...,...
995,996,2.62,"-0,0",0.00,2.62,19.24,16.62,5.27,1
996,997,19.24,"-0,0",0.00,19.24,14.88,-4.36,5.99,1
997,998,14.88,"-0,0",0.00,14.88,9.37,-5.51,-4.99,1
998,999,9.37,"-0,0",0.00,9.37,3.89,-5.48,-11.62,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,18449.36,"-6030,0",6911.24,19330.60,19460.91,130.31,181.93,0
1,2,19460.91,"-6125,0",8022.82,21358.73,21278.01,-80.72,73.05,0
2,3,21278.01,"-6065,0",6947.29,22160.30,22222.18,61.88,147.32,0
3,4,22222.18,"-5980,0",7979.31,24221.49,24182.24,-39.25,-32.62,0
4,5,24182.24,"-6245,0",2862.55,20799.79,20848.11,48.32,85.87,0
...,...,...,...,...,...,...,...,...,...
995,996,5.66,"-0,0",0.00,5.66,7.30,1.64,-9.82,1
996,997,7.30,"-0,0",0.00,7.30,10.34,3.04,-0.58,1
997,998,10.34,"-0,0",0.00,10.34,12.28,1.94,-5.24,1
998,999,12.28,"-0,0",0.00,12.28,8.88,-3.40,2.08,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22143.65,"-6055,0",6215.36,22304.01,22242.32,-61.69,34.11,0
1,2,22242.32,"-5940,0",6691.65,22993.97,22983.66,-10.31,42.41,0
2,3,22983.66,"-5885,0",6130.98,23229.64,23301.88,72.24,58.48,0
3,4,23301.88,"-6030,0",5035.77,22307.65,22270.88,-36.77,25.47,0
4,5,22270.88,"-6005,0",6518.15,22784.03,22794.42,10.39,13.96,0
...,...,...,...,...,...,...,...,...,...
995,996,2.74,"-0,0",0.00,2.74,3.88,1.14,3.00,1
996,997,3.88,"-0,0",0.00,3.88,0.72,-3.16,-2.72,1
997,998,0.72,"-0,0",0.00,0.72,1.13,0.41,-11.02,1
998,999,1.13,"-0,0",0.00,1.13,2.14,1.01,-19.34,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,15886.17,"-5900,0",5970.30,15956.47,15980.01,23.54,68.46,0
1,2,15980.01,"-5810,0",6050.74,16220.75,16175.65,-45.10,8.64,0
2,3,16175.65,"-5760,0",5917.51,16333.16,16413.96,80.80,17.30,0
3,4,16413.96,"-5970,0",6982.80,17426.76,17460.02,33.26,68.23,0
4,5,17460.02,"-6340,0",7929.64,19049.66,19111.41,61.75,52.84,0
...,...,...,...,...,...,...,...,...,...
995,996,5.52,"-0,0",0.00,5.52,0.47,-5.05,-4.84,1
996,997,0.47,"-0,0",0.00,0.47,5.79,5.32,0.68,1
997,998,5.79,"-0,0",0.00,5.79,14.14,8.35,-0.25,1
998,999,14.14,"-0,0",0.00,14.14,4.73,-9.41,2.66,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,19999.39,"-5905,0",6970.79,21065.18,21102.90,37.72,109.13,0
1,2,21102.90,"-6155,0",6060.28,21008.18,20966.69,-41.49,59.33,0
2,3,20966.69,"-5975,0",2934.69,17926.38,17990.19,63.81,85.10,0
3,4,17990.19,"-6305,0",6950.94,18636.13,18679.74,43.61,127.56,0
4,5,18679.74,"-5940,0",7943.20,20682.94,20731.84,48.90,178.79,0
...,...,...,...,...,...,...,...,...,...
995,996,5.73,"-0,0",0.00,5.73,3.13,-2.60,-19.24,1
996,997,3.13,"-0,0",0.00,3.13,3.87,0.74,-3.10,1
997,998,3.87,"-0,0",0.00,3.87,5.88,2.01,5.53,1
998,999,5.88,"-0,0",0.00,5.88,2.22,-3.66,-1.13,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22682.46,"-5995,0",5831.75,22519.21,22535.33,16.12,459.10,0
1,2,22535.33,"-5975,0",3507.95,20068.28,20135.89,67.61,514.24,0
2,3,20135.89,"-6250,0",8020.71,21906.60,21854.08,-52.52,421.71,0
3,4,21854.08,"-5985,0",8129.34,23998.42,23900.39,-98.03,282.93,0
4,5,23900.39,"-6030,0",5690.89,23561.28,23581.50,20.22,287.66,0
...,...,...,...,...,...,...,...,...,...
995,996,6.96,"-0,0",0.00,6.96,3.42,-3.54,-0.25,1
996,997,3.42,"-0,0",0.00,3.42,18.91,15.49,16.44,1
997,998,18.91,"-0,0",0.00,18.91,16.72,-2.19,9.09,1
998,999,16.72,"-0,0",0.00,16.72,1.44,-15.28,-0.25,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22379.58,"-5785,0",5690.39,22284.97,22319.50,34.53,144.59,0
1,2,22319.50,"-5980,0",6499.32,22838.82,22799.39,-39.43,171.58,0
2,3,22799.39,"-6000,0",6305.86,23105.25,23145.46,40.21,213.48,0
3,4,23145.46,"-6095,0",5814.71,22865.17,22845.93,-19.24,195.21,0
4,5,22845.93,"-6145,0",6270.76,22971.69,22978.10,6.41,290.05,0
...,...,...,...,...,...,...,...,...,...
995,996,2.71,"-0,0",0.00,2.71,2.61,-0.10,-0.48,1
996,997,2.61,"-0,0",0.00,2.61,5.93,3.32,1.85,1
997,998,5.93,"-0,0",0.00,5.93,13.80,7.87,5.27,1
998,999,13.80,"-0,0",0.00,13.80,6.10,-7.70,-0.73,1


### Conjunto de datos de prueba

In [45]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,14217.87,-6025,8078.98,16271.85,16202.83,-69.02,29.76,0
1,2,16202.83,-5915,7993.50,18281.33,18287.13,5.80,103.63,0
2,3,18287.13,-6005,5997.67,18279.80,18294.11,14.31,27.16,0
3,4,18294.11,-6085,3013.11,15222.22,15202.29,-19.93,93.76,0
4,5,15202.29,-5800,7941.86,17344.15,17431.85,87.70,159.23,0
...,...,...,...,...,...,...,...,...,...
995,996,15406.65,-6030,4978.48,14355.13,14337.52,-17.61,61.26,1
996,997,14337.52,-6000,5721.51,14059.03,14009.35,-49.68,-44.29,1
997,998,14009.35,-6275,6313.19,14047.54,14181.99,134.45,161.32,1
998,999,14181.99,-6150,5914.01,13946.00,14007.40,61.40,181.88,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21149.68,"-5960,0",5940.46,21130.14,21204.78,74.64,255.00,0
1,2,21204.78,"-6315,0",7021.73,21911.51,21862.93,-48.58,164.23,0
2,3,21862.93,"-5890,0",3218.08,19191.01,19194.42,3.41,149.26,0
3,4,19194.42,"-6160,0",6739.31,19773.73,19802.52,28.79,127.48,0
4,5,19802.52,"-5960,0",6979.79,20822.31,20853.99,31.68,148.24,0
...,...,...,...,...,...,...,...,...,...
995,996,2228.36,"-5465,58",4141.29,904.07,873.35,-30.72,-173.30,1
996,997,873.35,"-6090,0",6592.68,1376.03,1220.09,-155.94,-308.80,1
997,998,1220.09,"-5980,0",7181.83,2421.92,2463.95,42.03,-206.86,1
998,999,2463.95,"-5850,0",5758.71,2372.66,2400.72,28.06,-176.26,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,23654.87,"-6005,0",4298.93,21948.80,21972.61,23.81,163.31,0
1,2,21972.61,"-6095,0",4622.96,20500.57,20617.61,117.04,322.76,0
2,3,20617.61,"-6175,0",9067.79,23510.40,23442.51,-67.89,221.84,0
3,4,23442.51,"-5990,0",3410.32,20862.83,20890.70,27.87,233.22,0
4,5,20890.70,"-5915,0",6810.38,21786.08,21803.84,17.76,215.53,0
...,...,...,...,...,...,...,...,...,...
995,996,10.65,"-0,0",0.00,10.65,30.77,20.12,26.54,1
996,997,30.77,"-0,0",0.00,30.77,0.47,-30.30,-3.40,1
997,998,0.47,"-0,0",0.00,0.47,4.08,3.61,0.87,1
998,999,4.08,"-0,0",0.00,4.08,1.82,-2.26,-3.71,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21287.51,"-6070,0",7042.91,22260.42,22216.71,-43.71,188.20,0
1,2,22216.71,"-6065,0",4682.57,20834.28,20856.74,22.46,259.79,0
2,3,20856.74,"-6035,0",7234.09,22055.83,22134.30,78.47,272.27,0
3,4,22134.30,"-6120,0",5847.84,21862.14,22001.79,139.65,380.74,0
4,5,22001.79,"-5605,0",3757.23,20154.02,20186.77,32.75,279.92,0
...,...,...,...,...,...,...,...,...,...
995,996,3.31,"-0,0",0.00,3.31,4.22,0.91,-5.75,1
996,997,4.22,"-0,0",0.00,4.22,14.30,10.08,7.99,1
997,998,14.30,"-0,0",0.00,14.30,12.66,-1.64,10.33,1
998,999,12.66,"-0,0",0.00,12.66,2.96,-9.70,-1.54,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17548.09,"-6265,0",4958.31,16241.40,16245.87,4.47,160.94,0
1,2,16245.87,"-5725,0",6084.29,16605.16,16550.86,-54.30,43.48,0
2,3,16550.86,"-5880,0",6957.27,17628.13,17694.10,65.97,93.95,0
3,4,17694.10,"-6025,0",7912.37,19581.47,19651.53,70.06,229.90,0
4,5,19651.53,"-5880,0",5996.13,19767.66,19771.94,4.28,319.27,0
...,...,...,...,...,...,...,...,...,...
995,996,9.49,"-0,0",0.00,9.49,2.52,-6.97,-20.78,1
996,997,2.52,"-0,0",0.00,2.52,1.60,-0.92,-5.93,1
997,998,1.60,"-0,0",0.00,1.60,7.92,6.32,-9.84,1
998,999,7.92,"-0,0",0.00,7.92,1.37,-6.55,-12.18,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,16942.12,"-5825,0",5491.42,16608.54,16673.87,65.33,187.29,0
1,2,16673.87,"-5825,0",6897.10,17745.97,17875.86,129.89,237.48,0
2,3,17875.86,"-5965,0",6950.25,18861.11,18894.66,33.55,253.19,0
3,4,18894.66,"-5970,0",7962.56,20887.22,20937.92,50.70,387.03,0
4,5,20937.92,"-6135,0",5924.27,20727.19,20818.82,91.63,502.42,0
...,...,...,...,...,...,...,...,...,...
995,996,19.68,"-0,0",0.00,19.68,8.25,-11.43,0.72,1
996,997,8.25,"-0,0",0.00,8.25,3.65,-4.60,-8.36,1
997,998,3.65,"-0,0",0.00,3.65,8.40,4.75,4.92,1
998,999,8.40,"-0,0",0.00,8.40,23.49,15.09,16.35,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,18109.75,"-5940,0",6999.66,19169.41,19140.47,-28.94,290.50,0
1,2,19140.47,"-6320,0",7014.09,19834.56,19825.75,-8.81,346.46,0
2,3,19825.75,"-6145,0",7026.85,20707.60,20677.99,-29.61,326.59,0
3,4,20677.99,"-5805,0",4783.79,19656.78,19702.96,46.18,340.89,0
4,5,19702.96,"-6195,0",6592.28,20100.24,20021.69,-78.55,117.34,0
...,...,...,...,...,...,...,...,...,...
995,996,3.39,"-0,0",0.00,3.39,8.94,5.55,-9.07,1
996,997,8.94,"-0,0",0.00,8.94,13.46,4.52,1.87,1
997,998,13.46,"-0,0",0.00,13.46,5.20,-8.26,4.92,1
998,999,5.20,"-0,0",0.00,5.20,13.18,7.98,-1.52,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,23056.29,"-5975,0",3639.31,20720.60,20716.68,-3.92,334.01,0
1,2,20716.68,"-6150,0",7008.88,21575.56,21681.55,105.99,352.59,0
2,3,21681.55,"-6030,0",6463.74,22115.29,22138.84,23.55,354.54,0
3,4,22138.84,"-6330,0",4019.40,19828.24,19807.63,-20.61,339.90,0
4,5,19807.63,"-6315,0",10039.40,23532.03,23487.69,-44.34,121.91,0
...,...,...,...,...,...,...,...,...,...
995,996,1.75,"-0,0",0.00,1.75,13.26,11.51,10.31,1
996,997,13.26,"-0,0",0.00,13.26,4.60,-8.66,-5.13,1
997,998,4.60,"-0,0",0.00,4.60,9.65,5.05,6.75,1
998,999,9.65,"-0,0",0.00,9.65,6.10,-3.55,-3.36,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,23299.15,"-5900,0",4046.57,21445.72,21419.26,-26.46,496.31,0
1,2,21419.26,"-6310,0",8439.67,23548.93,23527.40,-21.53,356.96,0
2,3,23527.40,"-6055,0",7093.15,24565.55,24485.13,-80.42,233.55,0
3,4,24485.13,"-5925,0",2956.05,21516.18,21525.33,9.15,216.75,0
4,5,21525.33,"-6095,0",6018.92,21449.25,21425.77,-23.48,121.03,0
...,...,...,...,...,...,...,...,...,...
995,996,3.16,"-0,0",0.00,3.16,2.82,-0.34,-5.42,1
996,997,2.82,"-0,0",0.00,2.82,2.05,-0.77,1.14,1
997,998,2.05,"-0,0",0.00,2.05,1.39,-0.66,-3.80,1
998,999,1.39,"-0,0",0.00,1.39,10.38,8.99,-2.67,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,11544.40,"-5565,0",7069.47,13048.87,12982.38,-66.49,118.80,0
1,2,12982.38,"-6155,0",6953.56,13780.94,13840.48,59.54,124.42,0
2,3,13840.48,"-6005,0",8211.32,16046.80,15937.63,-109.17,-18.99,0
3,4,15937.63,"-5915,0",8550.06,18572.69,18595.78,23.09,76.99,0
4,5,18595.78,"-5830,0",6389.87,19155.65,19072.47,-83.18,-24.06,0
...,...,...,...,...,...,...,...,...,...
995,996,16.15,"-0,0",0.00,16.15,6.29,-9.86,-11.87,1
996,997,6.29,"-0,0",0.00,6.29,14.36,8.07,4.38,1
997,998,14.36,"-0,0",0.00,14.36,2.38,-11.98,-20.25,1
998,999,2.38,"-0,0",0.00,2.38,4.75,2.37,-6.32,1


<!-- TODO -->

## Selección de características

In [46]:
for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        data_dict[case][dataset] = data_dict[case][dataset].filter(items=['Variacion', 'Fugando combustible'])

### Conjunto de datos de entrenamiento

In [47]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Variacion,Fugando combustible
0,6.90,0
1,-16.05,0
2,58.65,0
3,54.55,0
4,0.40,0
...,...,...
995,47.79,1
996,1.07,1
997,-99.10,1
998,-7.15,1


,Variacion,Fugando combustible
0,44.62,0
1,14.74,0
2,15.51,0
3,-8.27,0
4,42.47,0
...,...,...
995,-121.42,1
996,-37.02,1
997,41.78,1
998,41.88,1


,Variacion,Fugando combustible
0,12.60,0
1,-94.41,0
2,6.56,0
3,42.94,0
4,36.04,0
...,...,...
995,-13.61,1
996,-4.39,1
997,11.38,1
998,9.09,1


,Variacion,Fugando combustible
0,-19.98,0
1,3.06,0
2,-17.23,0
3,153.68,0
4,36.46,0
...,...,...
995,16.62,1
996,-4.36,1
997,-5.51,1
998,-5.48,1


,Variacion,Fugando combustible
0,130.31,0
1,-80.72,0
2,61.88,0
3,-39.25,0
4,48.32,0
...,...,...
995,1.64,1
996,3.04,1
997,1.94,1
998,-3.40,1


...

,Variacion,Fugando combustible
0,-61.69,0
1,-10.31,0
2,72.24,0
3,-36.77,0
4,10.39,0
...,...,...
995,1.14,1
996,-3.16,1
997,0.41,1
998,1.01,1


,Variacion,Fugando combustible
0,23.54,0
1,-45.10,0
2,80.80,0
3,33.26,0
4,61.75,0
...,...,...
995,-5.05,1
996,5.32,1
997,8.35,1
998,-9.41,1


,Variacion,Fugando combustible
0,37.72,0
1,-41.49,0
2,63.81,0
3,43.61,0
4,48.90,0
...,...,...
995,-2.60,1
996,0.74,1
997,2.01,1
998,-3.66,1


,Variacion,Fugando combustible
0,16.12,0
1,67.61,0
2,-52.52,0
3,-98.03,0
4,20.22,0
...,...,...
995,-3.54,1
996,15.49,1
997,-2.19,1
998,-15.28,1


,Variacion,Fugando combustible
0,34.53,0
1,-39.43,0
2,40.21,0
3,-19.24,0
4,6.41,0
...,...,...
995,-0.10,1
996,3.32,1
997,7.87,1
998,-7.70,1


### Conjunto de datos de prueba

In [48]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Variacion,Fugando combustible
0,-69.02,0
1,5.80,0
2,14.31,0
3,-19.93,0
4,87.70,0
...,...,...
995,-17.61,1
996,-49.68,1
997,134.45,1
998,61.40,1


,Variacion,Fugando combustible
0,74.64,0
1,-48.58,0
2,3.41,0
3,28.79,0
4,31.68,0
...,...,...
995,-30.72,1
996,-155.94,1
997,42.03,1
998,28.06,1


,Variacion,Fugando combustible
0,23.81,0
1,117.04,0
2,-67.89,0
3,27.87,0
4,17.76,0
...,...,...
995,20.12,1
996,-30.30,1
997,3.61,1
998,-2.26,1


,Variacion,Fugando combustible
0,-43.71,0
1,22.46,0
2,78.47,0
3,139.65,0
4,32.75,0
...,...,...
995,0.91,1
996,10.08,1
997,-1.64,1
998,-9.70,1


,Variacion,Fugando combustible
0,4.47,0
1,-54.30,0
2,65.97,0
3,70.06,0
4,4.28,0
...,...,...
995,-6.97,1
996,-0.92,1
997,6.32,1
998,-6.55,1


...

,Variacion,Fugando combustible
0,65.33,0
1,129.89,0
2,33.55,0
3,50.70,0
4,91.63,0
...,...,...
995,-11.43,1
996,-4.60,1
997,4.75,1
998,15.09,1


,Variacion,Fugando combustible
0,-28.94,0
1,-8.81,0
2,-29.61,0
3,46.18,0
4,-78.55,0
...,...,...
995,5.55,1
996,4.52,1
997,-8.26,1
998,7.98,1


,Variacion,Fugando combustible
0,-3.92,0
1,105.99,0
2,23.55,0
3,-20.61,0
4,-44.34,0
...,...,...
995,11.51,1
996,-8.66,1
997,5.05,1
998,-3.55,1


,Variacion,Fugando combustible
0,-26.46,0
1,-21.53,0
2,-80.42,0
3,9.15,0
4,-23.48,0
...,...,...
995,-0.34,1
996,-0.77,1
997,-0.66,1
998,8.99,1


,Variacion,Fugando combustible
0,-66.49,0
1,59.54,0
2,-109.17,0
3,23.09,0
4,-83.18,0
...,...,...
995,-9.86,1
996,8.07,1
997,-11.98,1
998,2.37,1


<!-- TODO -->

## Unificación de conjunto de datos

In [49]:
complete_train_data = pd.concat([element['train_dataframe'] for element in data_dict.values()])

train_data = (
    complete_train_data.drop(columns='Fugando combustible'), 
    complete_train_data['Fugando combustible']
)

In [50]:
segmented_test_data = [
    (element['leak_value'], element['test_dataframe']) for element in data_dict.values()
]

test_data = [(
    'general',
    pd.concat([element['test_dataframe'] for element in data_dict.values()], ignore_index=True)
)]

## Implementación de modelos

### Definición de métricas

In [51]:
def leak_detection_delay(y_true: np.array, y_pred: np.array) -> int:
    real_first_leak_day = 0
    while y_true[real_first_leak_day] != 1: real_first_leak_day += 1

    pred_first_leak_day = real_first_leak_day
    while y_pred[pred_first_leak_day] != 1: pred_first_leak_day += 1

    return (pred_first_leak_day - real_first_leak_day)

In [52]:
def performance_metrics(y_true, y_pred, leak_value = 0.013) -> dict:
    return {
        # Metricas clasicas de ML
            'accuracy_score':   accuracy_score(y_true, y_pred),
            'f1_score':         f1_score(y_true, y_pred),
            'recall_score':     recall_score(y_true, y_pred),
            'precision_score':  precision_score(y_true, y_pred),
            'confusion_matrix': confusion_matrix(y_true, y_pred),
        # Metricas propias del problema
            'false_positives':  confusion_matrix(y_true, y_pred).ravel()[1],
            'leak_delay':       leak_detection_delay(y_true, y_pred),
            'leak_loss':        leak_detection_delay(y_true, y_pred) * leak_value * 60 * 24
    }

### Modelos clásicos

In [53]:
from sklearn.model_selection import GridSearchCV
import pickle as pkl

param_grids_path = '../../pkl/param_grids'

def train_classic_model(classifier_class, param_grid, train_data, cv = 5):
    param_pkl_path = f'{param_grids_path}/{classifier_class.__name__}.pkl'
    classifier = None
    X_train, y_train = train_data
    if os.path.exists(param_pkl_path):  # Param grid already known
        param_grid = pkl.load(open(param_pkl_path, 'rb'))
        classifier = classifier_class(**param_grid)
        classifier.fit(X_train, y_train)
    else:
        grid_search = GridSearchCV(
            estimator = classifier_class(),
            param_grid = param_grid,
            cv = cv
        )
        grid_search.fit(X_train, y_train)
        classifier = grid_search.best_estimator_
        pkl.dump(
            obj = grid_search.best_params_,
            file = open(param_pkl_path, 'wb')
        )
    return classifier

In [54]:
def test_classic_model(classifier, test_list) -> list[dict]:
    results = list()
    for (leak_value, test_data) in test_list:
        X_test = test_data.drop(columns='Fugando combustible')
        y_test = test_data['Fugando combustible']
        y_pred = classifier.predict(X_test)
        results.append((
            leak_value,
            performance_metrics(y_test, y_pred, leak_value)
        ))
    return results

##### Random Forest Classifier

In [55]:
from sklearn.ensemble import RandomForestClassifier

rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = train_classic_model(RandomForestClassifier, rf_param_grid, train_data)

rf_seg_results = test_classic_model(rf_model, segmented_test_data)
rf_results = test_classic_model(rf_model, test_data)

##### Naive Bayes Classifier

In [20]:
from sklearn.naive_bayes import GaussianNB

gnb_param_grid = {
    'var_smoothing': [1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

gnb_model = train_classic_model(GaussianNB, gnb_param_grid, train_data)

gnb_seg_results = test_classic_model(gnb_model, segmented_test_data)
gnb_results = test_classic_model(gnb_model, test_data)

##### KNN Classifier

In [23]:
from sklearn.neighbors import KNeighborsClassifier

knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_model = train_classic_model(KNeighborsClassifier, knn_param_grid, train_data)

knn_seg_results = test_classic_model(knn_model, segmented_test_data)
knn_results = test_classic_model(knn_model, test_data)

##### XGBoost Classifier

In [26]:
from xgboost import XGBClassifier

xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

xgb_model = train_classic_model(XGBClassifier, xgb_param_grid, train_data)

xgb_seg_results = test_classic_model(xgb_model, segmented_test_data)
xgb_results = test_classic_model(xgb_model, test_data)

##### _Ensemble Classifier_

In [29]:
from sklearn.ensemble import VotingClassifier

clf_model = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier()),
        ('nb', GaussianNB()),
        ('knn', KNeighborsClassifier()),
        ('XGB', XGBClassifier())
    ],
    voting='hard'
)

clf_model.fit(train_data[0], train_data[1])

clf_seg_results = test_classic_model(clf_model, segmented_test_data)
clf_results = test_classic_model(clf_model, test_data)

### Modelos _Deep Learning_

In [32]:
def eval_dl_model(model, test_list):
    result_dict = {}
    for leak_value, test_dataframe in test_list:
        X_test = test_dataframe.drop(columns='Fugando combustible')
        y_test = test_dataframe['Fugando combustible']
        
        y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

        accuracy = accuracy_score(y_test, y_pred)
        leak_delay = leak_detection_delay(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        leak_loss = (leak_value * 60 * 24 * leak_delay) if type(leak_value) == float else 0

        result_dict[leak_value] = (accuracy, f1, fp, leak_delay, leak_loss)

    return pd.DataFrame.from_dict(
        data = result_dict,
        orient = 'index',
        columns = ['accuracy_score', 'f1_score', 'false positives', 'leak_detection_delay', 'leak_loss']
    )

#### Único modelo de _Deep Learning_

In [33]:
X_train, y_train = train_data

##### Modelo unicapa

In [34]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(15, dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x = X_train,
    y = y_train,
    epochs = 50,
    batch_size = 8192,
    verbose = 0
)

display(eval_dl_model(model, segmented_test_data))

,accuracy_score,f1_score,false positives,leak_detection_delay,leak_loss
0.013000,0.508,0.385000,141,0,0.000000
0.054929,0.529,0.406053,127,4,316.388571
0.096857,0.603,0.548350,133,1,139.474286
0.138786,0.702,0.702595,145,1,199.851429
0.180714,0.750,0.761450,144,0,0.000000
0.222643,0.787,0.801120,137,0,0.000000
0.264571,0.800,0.817185,142,0,0.000000
0.306500,0.820,0.837251,138,0,0.000000
0.348429,0.830,0.848754,142,0,0.000000
0.390357,0.818,0.836625,143,0,0.000000


##### Modelo multicapa

In [35]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(25, return_sequences=True, dropout=0.2),
    tf.keras.layers.LSTM(40, return_sequences=True, dropout=0.2),
    tf.keras.layers.LSTM(60, dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x = X_train,
    y = y_train,
    epochs = 50,
    batch_size = 8192,
    verbose = 0
)

display(eval_dl_model(model, segmented_test_data))

,accuracy_score,f1_score,false positives,leak_detection_delay,leak_loss
0.013000,0.525,0.429772,149,0,0.000000
0.054929,0.549,0.470035,146,4,316.388571
0.096857,0.620,0.594017,153,0,0.000000
0.138786,0.712,0.724138,161,1,199.851429
0.180714,0.758,0.778388,162,0,0.000000
0.222643,0.791,0.810517,151,0,0.000000
0.264571,0.792,0.815275,162,0,0.000000
0.306500,0.814,0.836268,156,0,0.000000
0.348429,0.827,0.848644,153,0,0.000000
0.390357,0.820,0.842105,155,0,0.000000


#### Múltiples modelos de _Deep Learning_

In [36]:
results = {}
for case in data_dict:
    X_train = data_dict[case]['train_dataframe'].drop(columns='Fugando combustible')
    y_train = data_dict[case]['train_dataframe']['Fugando combustible']
    
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
        tf.keras.layers.LSTM(15, dropout=0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        x = X_train,
        y = y_train,
        epochs = 50,
        batch_size = 8192,
        verbose = 0
    )

    leak_value = data_dict[case]['leak_value']

    X_test = data_dict[case]['test_dataframe'].drop(columns='Fugando combustible')
    y_test = data_dict[case]['test_dataframe']['Fugando combustible']
    
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

    accuracy = accuracy_score(y_test, y_pred)
    leak_delay = leak_detection_delay(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    leak_loss = (leak_value * 60 * 24 * leak_delay) if type(leak_value) == float else 0

    results[leak_value] = (accuracy, f1, fp, leak_delay, leak_loss)
    # model.save(f'../../pkl/models/{case}.keras')

display(pd.DataFrame.from_dict(
    data = results,
    orient = 'index',
    columns = ['accuracy_score', 'f1_score', 'false positives', 'leak_detection_delay', 'leak_loss']
))

,accuracy_score,f1_score,false positives,leak_detection_delay,leak_loss
0.013000,0.497,0.564502,324,1,18.720000
0.054929,0.638,0.663569,214,1,79.097143
0.096857,0.657,0.702515,243,0,0.000000
0.138786,0.719,0.769104,244,0,0.000000
0.180714,0.586,0.627698,258,0,0.000000
0.222643,0.724,0.771144,236,0,0.000000
0.264571,0.677,0.710833,215,0,0.000000
0.306500,0.554,0.640323,338,0,0.000000
0.348429,0.628,0.712519,328,0,0.000000
0.390357,0.705,0.755187,245,0,0.000000


## Visualización de resultados

In [ ]:
def display_performance_table(results: list[dict]) -> None:
    display(pd.DataFrame.from_records(results).drop(columns='confusion_matrix'))